In [5]:
!pip install yfinance fpdf --quiet

  Preparing metadata (setup.py) ... done


In [25]:

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def get_value_by_partial_label(df, label):
    for index in df.index:
        if label.lower() in index.lower():
            return df.loc[index][0]
    return None

ticker = "AEGISLOG.NS"
stock = yf.Ticker(ticker)

cashflow = stock.cashflow
balance_sheet = stock.balance_sheet
income_statement = stock.financials
info = stock.info

print("Income Statement Items:", income_statement.index.tolist())
print("Cash Flow Items:", cashflow.index.tolist())
print("Balance Sheet Items:", balance_sheet.index.tolist())

revenue = get_value_by_partial_label(income_statement, 'Total Revenue')
ebit = get_value_by_partial_label(income_statement, 'EBIT')
ocf = get_value_by_partial_label(cashflow, 'Operating Cash Flow')
capex = get_value_by_partial_label(cashflow, 'Capital Expenditure')





if None in [revenue, ebit, ocf, capex]:
    raise ValueError("Missing financial data!")

fcf = ocf - abs(capex)

total_debt = get_value_by_partial_label(balance_sheet, 'Total Debt')
equity = get_value_by_partial_label(balance_sheet, 'Stockholders Equity')
interest_expense = get_value_by_partial_label(income_statement, 'Interest Expense')
current_assets = get_value_by_partial_label(balance_sheet, 'Current Assets')
current_liabilities = get_value_by_partial_label(balance_sheet, 'Current Liabilities')

current_ratio = current_assets / current_liabilities if current_liabilities else None
debt_to_equity = total_debt / equity if equity else None
interest_coverage = ebit / interest_expense if interest_expense else None
roe = (ebit / equity) if equity else None

print(f"📌 Debt-to-Equity Ratio: {debt_to_equity:.2f}")
print(f"📌 Interest Coverage Ratio: {interest_coverage:.2f}")
print(f"📌 Current Ratio: {current_ratio:.2f}")
print(f"📌 Return on Equity (ROE): {roe:.2%}")

# --- DCF Assumptions ---
growth_rate = 0.08
discount_rate = 0.12
years = 5

scenarios = {
    "Base": growth_rate,
    "Optimistic": growth_rate + 0.02,
    "Pessimistic": growth_rate - 0.02
}

results = {}

for scenario, g_rate in scenarios.items():
    future_fcf = [fcf * ((1 + g_rate) ** i) for i in range(1, years + 1)]
    terminal_value = future_fcf[-1] * (1 + g_rate) / (discount_rate - g_rate)

    discounted_fcf = [fcf / ((1 + discount_rate) ** i) for i, fcf in enumerate(future_fcf, 1)]
    discounted_terminal = terminal_value / ((1 + discount_rate) ** years)

    enterprise_value = sum(discounted_fcf) + discounted_terminal

    cash = get_value_by_partial_label(balance_sheet, 'Cash')
    debt = get_value_by_partial_label(balance_sheet, 'Total Debt')
    cash = 0 if cash is None else cash
    debt = 0 if debt is None else debt
    equity_value = max(enterprise_value + cash - debt, 0)
    shares_outstanding = info.get("sharesOutstanding", 0)

    if shares_outstanding == 0:
        raise ValueError("Shares outstanding not available!")

    intrinsic_value = equity_value / shares_outstanding
    results[scenario] = intrinsic_value

discount_rates = [0.10, 0.12, 0.14]
growth_rates = [0.05, 0.08, 0.10]
sensitivity_matrix = pd.DataFrame(index=[f"{gr*100:.0f}%" for gr in growth_rates], columns=[f"{dr*100:.0f}%" for dr in discount_rates])

for gr in growth_rates:
    for dr in discount_rates:
        if dr == gr:
            sensitivity_matrix.loc[f"{gr*100:.0f}%", f"{dr*100:.0f}%"] = "NaN"
            continue

        future_fcf = [fcf * ((1 + gr) ** i) for i in range(1, years + 1)]
        terminal_value = future_fcf[-1] * (1 + gr) / (dr - gr)
        discounted_fcf = [fcf / ((1 + dr) ** i) for i, fcf in enumerate(future_fcf, 1)]
        discounted_terminal = terminal_value / ((1 + dr) ** years)
        ev = sum(discounted_fcf) + discounted_terminal
        eq_val = ev + cash - debt
        intrinsic = eq_val / shares_outstanding
        sensitivity_matrix.loc[f"{gr*100:.0f}%", f"{dr*100:.0f}%"] = round(intrinsic, 2)

# --- Comparative Analysis ---
peers = ["RELIANCE.NS", "BPCL.NS", "IOC.NS"]
comparative_data = []

for peer in peers:
    try:
        peer_info = yf.Ticker(peer).info
        comparative_data.append({
            "Company": peer,
            "P/E": peer_info.get("trailingPE"),
            "EV/EBITDA": peer_info.get("enterpriseToEbitda"),
            "ROE": peer_info.get("returnOnEquity"),
            "ROA": peer_info.get("returnOnAssets")
        })
    except:
        continue

peer_df = pd.DataFrame(comparative_data)

# --- Enhanced Markdown Report Generation ---
md_content = f"""# 📘 DCF & Financial Analysis Report – Aegis Logistics Ltd. (AEGISLOG.NS)

**Date:** {pd.Timestamp.today().strftime('%Y-%m-%d')}
**Objective:** To evaluate the intrinsic value and financial health of Aegis Logistics using DCF, risk assessment, and benchmarking.

---

## 🏢 Company Overview

- **Company Name:** Aegis Logistics Ltd.
- **Sector:** {info.get('sector', 'N/A')}
- **Industry:** {info.get('industry', 'N/A')}
- **Market Cap:** ₹{info.get('marketCap', 'N/A'):,}
- **Current Price:** ₹{info.get('currentPrice', 'N/A')}

---

## 💰 Key Financial Data

| Metric                    | Value (₹)      |
|---------------------------|----------------|
| Operating Cash Flow (OCF) | {ocf:,.0f}     |
| Capital Expenditure       | {capex:,.0f}   |
| Free Cash Flow (FCF)      | {fcf:,.0f}     |
| Total Debt                | {debt:,.0f}    |
| Cash & Equivalents        | {cash:,.0f}    |
| Shareholders' Equity      | {equity:,.0f}  |

---

## 📉 Credit Risk & Financial Ratios

| Ratio                     | Value           |
|---------------------------|-----------------|
| Debt-to-Equity Ratio      | {debt_to_equity:.2f}x       |
| Interest Coverage Ratio   | {interest_coverage:.2f}x    |
| Return on Equity (ROE)    | {roe:.2%}        |

*These ratios reflect Aegis Logistics’ financial stability and creditworthiness.*

---

## 📈 DCF Valuation (Base Case)

**Growth Rate:** {growth_rate*100:.1f}%
**Discount Rate:** {discount_rate*100:.1f}%
**Projection Period:** {years} years

| Year | Projected FCF (₹) | Discounted FCF (₹) |
|------|------------------|-------------------|
"""
for i, (fcf_val, disc_fcf_val) in enumerate(zip(future_fcf, discounted_fcf), 1):
    md_content += f"| {i} | {fcf_val:,.0f} | {disc_fcf_val:,.0f} |\n"

md_content += f"""
- **Terminal Value:** ₹{terminal_value:,.0f}
- **Discounted Terminal Value:** ₹{discounted_terminal:,.0f}
- **Enterprise Value (EV):** ₹{enterprise_value:,.0f}
- **Equity Value:** ₹{equity_value:,.0f}
- **Intrinsic Value per Share:** ₹{intrinsic_value:.2f}

---

## 🧪 Scenario Analysis – Intrinsic Value per Share

| Scenario    | Growth Rate | Intrinsic Value (₹) |
|-------------|-------------|----------------------|
"""

for scenario, g_rate in scenarios.items():
    md_content += f"| {scenario} | {g_rate*100:.1f}% | ₹{results[scenario]:.2f} |\n"
"""
---

## 🔁 Sensitivity Analysis (Growth Rate vs Discount Rate)

| Growth \\ Discount | {' | '.join(sensitivity_matrix.columns)} |
|--------------------|{''.join(['------------|' for _ in sensitivity_matrix.columns])}
"""
for gr_label in sensitivity_matrix.index:
    row = ' | '.join(str(sensitivity_matrix.loc[gr_label][col]) for col in sensitivity_matrix.columns)
    md_content += f"| {gr_label} | {row} |\n"
"""
---

## 🧮 Comparative Valuation (Benchmarking)

| Metric     | Aegis Logistics | Peer Average |
|------------|-----------------|--------------|
| EV/EBITDA  | {ev_ebitda:.2f}x | {peer_ev_ebitda:.2f}x |
| P/E Ratio  | {pe_ratio:.2f}x  | {peer_pe:.2f}x        |
| ROE        | {roe:.2%}        | {peer_roe:.2%}        |
| ROA        | {roa:.2%}        | {peer_roa:.2%}        |

*Comparison with industry peers helps contextualize the valuation and profitability of Aegis Logistics.*

---

## ✅ Conclusion

- The DCF-based **intrinsic value per share** is estimated between ₹{min(results.values()):.2f} and ₹{max(results.values()):.2f} depending on the growth and discount assumptions.
- Financial ratios suggest **moderate leverage** with stable operational cash flows.
- Benchmarking indicates the stock is **{('undervalued' if intrinsic_value > info.get('currentPrice', 0) else 'overvalued')}** compared to peers.

📌 *Recommendation:* Use this analysis in conjunction with qualitative factors and current market trends before investment decisions.

---

"""

with open("Enhanced_DCF_Report_AegisLogistics.md", "w", encoding="utf-8") as f:
    f.write(md_content)

print("✅ Enhanced Markdown Report Generated: Enhanced_DCF_Report_AegisLogistics.md")


Income Statement Items: ['Tax Effect Of Unusual Items', 'Tax Rate For Calcs', 'Normalized EBITDA', 'Total Unusual Items', 'Total Unusual Items Excluding Goodwill', 'Net Income From Continuing Operation Net Minority Interest', 'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA', 'EBIT', 'Net Interest Income', 'Interest Expense', 'Interest Income', 'Normalized Income', 'Net Income From Continuing And Discontinued Operation', 'Total Expenses', 'Rent Expense Supplemental', 'Diluted Average Shares', 'Basic Average Shares', 'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders', 'Net Income Common Stockholders', 'Otherunder Preferred Stock Dividend', 'Net Income', 'Minority Interests', 'Net Income Including Noncontrolling Interests', 'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income', 'Other Non Operating Income Expenses', 'Special Income Charges', 'Other Special Charges', 'Write Off', 'Net Non Operating Interest Income Expense', 'Total Other Financ

<ipython-input-25-7886ebb8f572>:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.loc[index][0]


✅ Enhanced Markdown Report Generated: Enhanced_DCF_Report_AegisLogistics.md
